# Tutorial 6: Advanced Usage - Working with SQL

<div class="alert alert-block alert-info"> <b>Before we get started: </b> 
    <ul style="list-style-type: none;margin: 0;padding: 0;">
        <li>✍️ To run this notebook, you need to have Ponder installed and set up on your machine. If you have not done so already, please refer to our <a href="https://docs.ponder.io/getting_started/quickstart.html">Quickstart guide</a> to get started.</li>
        <li>📖 Otherwise, if you're just interested in browsing through the tutorial, keep reading below!</li>
    </ul>
</div>

In this tutorial, we will showcase a few tips and tricks that helps you more easily move between Ponder and SQL. We will be using the [MIMIC-III demo dataset](https://physionet.org/content/mimiciii-demo/1.4/) as an example dataset. The MIMIC-III Clinical Database contains deidentified health-related data of patients who stayed in an intensive care unit (ICU) at the Beth Israel Deaconess Medical Center in Boston. The demo dataset contains records for 100 patients across three tables `PATIENTS`, `ICUSTAYS`, and `ADMISSIONS`. 

In [1]:
import os; os.chdir("..")

In [2]:
import ponder; ponder.init()
import modin.pandas as pd

2023-04-21 05:51:49,505 - <module ponder> - WARNING - New version of Ponder is available! `pip install --upgrade ponder` to install!


### Data Definition (DDL) with DuckDB

In SQL, DDL statements involve modifications to the database schema, e.g., `CREATE`, `ALTER`, `DROP`. Oftentimes, you may want to run a DDL statement alongside your analysis, either via an external query editor or through SQLAlchemy. 



You can reuse the same DuckDB connection used for Ponder to run any SQL queries.

In [3]:
import duckdb
duckdb_con = duckdb.connect("ponder.db")

In [4]:
ponder.configure(default_connection=duckdb_con)

Then we can run the SQL query directly to create a new database named `MIMIC3`

In [5]:
duckdb_con.execute("CREATE SCHEMA IF NOT EXISTS MIMIC3;") 

Now if we print out the database, we can see the new schema MIMIC3 added: 

In [6]:
duckdb_con.execute('SELECT * FROM information_schema.schemata;').df()

,catalog_name,schema_name,schema_owner,default_character_set_catalog,default_character_set_schema,default_character_set_name,sql_path
0,ponder,MIMIC3,duckdb,NaN,NaN,NaN,NaN
1,ponder,information_schema,duckdb,NaN,NaN,NaN,NaN
2,ponder,main,duckdb,NaN,NaN,NaN,NaN
3,ponder,pg_catalog,duckdb,NaN,NaN,NaN,NaN
4,system,information_schema,duckdb,NaN,NaN,NaN,NaN
5,system,main,duckdb,NaN,NaN,NaN,NaN
6,system,pg_catalog,duckdb,NaN,NaN,NaN,NaN
7,temp,information_schema,duckdb,NaN,NaN,NaN,NaN
8,temp,main,duckdb,NaN,NaN,NaN,NaN
9,temp,pg_catalog,duckdb,NaN,NaN,NaN,NaN


In DuckDB, the default schema is `main`, you can print out the list of tables in `main`:

In [7]:
duckdb_con.execute('SELECT * FROM duckdb_tables;').df()

,database_name,database_oid,schema_name,schema_oid,table_name,table_oid,internal,temporary,has_primary_key,estimated_size,column_count,index_count,check_constraint_count,sql
0,ponder,4,main,856,PONDER_CITIBIKE,878,False,False,False,118865,15,0,0,"CREATE TABLE ""PONDER_CITIBIKE""(tripduration DO..."
1,ponder,4,main,856,PONDER_BOOKS,876,False,False,False,11123,12,0,0,"CREATE TABLE ""PONDER_BOOKS""(""bookID"" BIGINT, t..."
2,ponder,4,main,856,PONDER_TAXI,874,False,False,False,210035,17,0,0,"CREATE TABLE ""PONDER_TAXI""(""VENDORID"" BIGINT, ..."
3,ponder,4,main,856,PONDER_CUSTOMER,872,False,False,False,100,8,0,0,"CREATE TABLE ""PONDER_CUSTOMER""(""C_CUSTKEY"" BIG..."
4,ponder,4,main,856,PONDER_ORDERS,870,False,False,False,145,9,0,0,"CREATE TABLE ""PONDER_ORDERS""(""O_ORDERKEY"" BIGI..."
5,ponder,4,main,856,PONDER_PART,868,False,False,False,3893,9,0,0,"CREATE TABLE ""PONDER_PART""(""P_PARTKEY"" BIGINT,..."
6,ponder,4,main,856,PONDER_SUPPLIER,866,False,False,False,3255,7,0,0,"CREATE TABLE ""PONDER_SUPPLIER""(""S_SUPPKEY"" BIG..."


Finally, you can close the DuckDB connection when you finish. 

In [8]:
duckdb_con.close()

### Existing SQL DML with Ponder

We will be using a few example tables for the remainder of this tutorial. You can run this python script to populate the required datasets to your database. This will populate three different tables `PATIENTS`, `ADMISSIONS`, and `ICUSTAYS` to your database. 

In [9]:
!python populate_mimic3.py

2023-04-21 05:52:07,305 - <module ponder> - WARNING - New version of Ponder is available! `pip install --upgrade ponder` to install!
Uploaded dataset to ICUSTAYS
Uploaded dataset to PATIENTS
Uploaded dataset to ADMISSIONS


In [10]:
import duckdb
duckdb_con = duckdb.connect("mimic3.db")
ponder.configure(default_connection=duckdb_con)

In [11]:
duckdb_con.execute('SELECT * FROM duckdb_tables;').df()

,database_name,database_oid,schema_name,schema_oid,table_name,table_oid,internal,temporary,has_primary_key,estimated_size,column_count,index_count,check_constraint_count,sql
0,mimic3,4,main,856,ICUSTAYS,870,False,False,False,136,12,0,0,"CREATE TABLE ""ICUSTAYS""(row_id BIGINT, subject..."
1,mimic3,4,main,856,PATIENTS,868,False,False,False,100,8,0,0,"CREATE TABLE ""PATIENTS""(row_id BIGINT, subject..."
2,mimic3,4,main,856,ADMISSIONS,866,False,False,False,129,19,0,0,"CREATE TABLE ""ADMISSIONS""(row_id BIGINT, subje..."


Oftentime, you may already have an existing SQL script that you've been using to join and denormalize some tables or perform some pre-aggregation or ETL before you perform your analysis. You want to reuse that SQL code while working with Ponder for the remaining analysis workflow. In this example, we show how you can feed this into the `pd.read_sql` to operate on the resulting table.

In [12]:
ponder.configure(default_connection=duckdb_con)

For example, we may want to use [this existing SQL query](https://mimic.mit.edu/docs/iii/tutorials/intro-to-mimic-iii/#5-patient-age-and-mortality) from the MIT MIMIC-III tutorial to jumpstart our analysis. 

In [13]:
df = pd.read_sql('''SELECT p."subject_id", p."dob", a."hadm_id",
                    a."admittime", p."expire_flag"
                    FROM ADMISSIONS as a
                    INNER JOIN PATIENTS as p
                    ON p."subject_id" = a."subject_id"''', con = duckdb_con)

In [14]:
df

,subject_id,dob,hadm_id,admittime,expire_flag
0,10006,2094-03-05 00:00:00,142345,2164-10-23 21:09:00,1
1,10011,2090-06-05 00:00:00,105331,2126-08-14 22:32:00,1
2,10013,2038-09-03 00:00:00,165520,2125-10-04 23:36:00,1
3,10017,2075-09-21 00:00:00,199207,2149-05-26 17:19:00,1
4,10019,2114-06-20 00:00:00,177759,2163-05-14 20:43:00,1
...,...,...,...,...,...
124,44083,2057-11-15 00:00:00,198330,2112-05-28 15:45:00,1
125,44154,1878-05-14 00:00:00,174245,2178-05-14 20:29:00,1
126,44212,2078-06-16 00:00:00,163189,2123-11-24 14:14:00,1
127,44222,2107-06-27 00:00:00,192189,2180-07-19 06:55:00,1


Then we can continue using Ponder by writing pandas as always.

In [15]:
df["dob"] = pd.to_datetime(df["dob"])
df["admittime"] = pd.to_datetime(df["admittime"])

In [16]:
df["age"] = df["admittime"].dt.year  - df["dob"].dt.year

In [17]:
df["age"]

0       70
1       36
2       87
3       74
4       49
      ... 
124     55
125    300
126     45
127     73
128     58
Name: age, Length: 129, dtype: int64

### Working with multiple tables

With Ponder, you can work with multiple tables at the same time by creating different dataframes using the `read_sql` or `read_csv` command.

In [18]:
patients = pd.read_sql("PATIENTS", con=duckdb_con)
admissions = pd.read_sql("ADMISSIONS", con=duckdb_con)

Now we can work with these two dataframes in pandas. Here, we perform the same query as the SQL query above: 
```sql
SELECT p."subject_id", p."dob", a."hadm_id",
       a."admittime", p."expire_flag"
       FROM ADMISSIONS as a
       INNER JOIN PATIENTS as p
       ON p."subject_id" = a."subject_id"
```

In [19]:
patients.merge(admissions,on="subject_id")[["subject_id", "dob", "hadm_id","admittime", "expire_flag"]]

,subject_id,dob,hadm_id,admittime,expire_flag
0,10006,2094-03-05 00:00:00,142345,2164-10-23 21:09:00,1
1,10011,2090-06-05 00:00:00,105331,2126-08-14 22:32:00,1
2,10013,2038-09-03 00:00:00,165520,2125-10-04 23:36:00,1
3,10017,2075-09-21 00:00:00,199207,2149-05-26 17:19:00,1
4,10019,2114-06-20 00:00:00,177759,2163-05-14 20:43:00,1
...,...,...,...,...,...
124,44083,2057-11-15 00:00:00,198330,2112-05-28 15:45:00,1
125,44154,1878-05-14 00:00:00,174245,2178-05-14 20:29:00,1
126,44212,2078-06-16 00:00:00,163189,2123-11-24 14:14:00,1
127,44222,2107-06-27 00:00:00,192189,2180-07-19 06:55:00,1


In [20]:
duckdb_con.close()